In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import math
import re

from nilearn import datasets, plotting
from nilearn import maskers, image
import matplotlib.pyplot as plt
import nibabel

Hurst exponent function definition:
    R/S method

In [ ]:
from hurst import compute_Hc 

def calculate_hurst(time_series):
    """
    Calculate the Hurst exponent for each voxel in the input 2D array.

    Parameters:
    - time_series: 2D array where each column is a time series of a specific voxel.

    Returns:
    - Array of Hurst exponents for each voxels.
    """

    hurst_exponent=0.0
    hurst_constant=0.0
    hurst_data=[]
    # for column in zip(*time_series):
    
    if(len(time_series) > 111):
        hurst_exponent, hurst_constant, _ = compute_Hc(time_series)
    return hurst_exponent,hurst_constant

Hurst exponent function definition:
    DFA method

In [ ]:
def calculate_window_sizes(series_length):
    if series_length < 112:
        window_sizes = [0]
    else:
        scale_lim=[3,5]
        scale_dens=0.20
        window_sizes=(2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(int)
    return window_sizes


def calc_rms(x, scale):
    shape = (x.shape[0] // scale, scale)
    X = np.lib.stride_tricks.as_strided(x, shape=shape)
    scale_ax = np.arange(scale)
    rms = np.zeros(X.shape[0])
    for e, xcut in enumerate(X):
        coeff = np.polyfit(scale_ax, xcut, 1)
        xfit = np.polyval(coeff, scale_ax)
        rms[e] = np.sqrt(np.mean((xcut - xfit)**2))
    return rms

def dfa( time_series_data,window_sizes,show=False):
    if len(window_sizes) != 1 :
        y = np.cumsum(time_series_data - np.mean(time_series_data))
        fluct = np.zeros(len(window_sizes))
        for e, sc in enumerate(window_sizes):
            fluct[e] = np.mean(np.sqrt(calc_rms(y, sc)**2))
        coeff = np.polyfit(np.log2(window_sizes), np.log2(fluct), 1)
        hurst_exponent = coeff[0]
        # Calculate R-squared
        predicted_values = np.polyval(coeff, np.log2(window_sizes))
        mean_fluct = np.mean(np.log2(fluct))
        tss = np.sum((np.log2(fluct) - mean_fluct) ** 2)
        rss = np.sum((np.log2(fluct) - predicted_values) ** 2)
        r_squared = 1 - (rss / tss)

        if show:
            fluctfit = 2**np.polyval(coeff, np.log2(window_sizes))
            plt.loglog(window_sizes, fluct, 'bo')
            plt.loglog(window_sizes, fluctfit, 'r', label=f'H = {coeff[0]:0.2f}, R² = {r_squared:.2f}')
            plt.title('Detrended Fluctuation Analysis')
            plt.xlabel(r'$\log_{2}$(time window)')
            plt.ylabel(r'$\log_{2}$<F(t)>')
            plt.legend()
            plt.show()

    else:
        hurst_exponent=0
        r_squared=0
    
    return hurst_exponent, r_squared

In [ ]:
def calculate_step_size(length, window_size):
    num_windows = math.ceil(length / window_size)
    if num_windows == 1:
        return window_size  # No overlap needed if the list is shorter than the window size
    
    step_size = max(1, (length - window_size) // (num_windows - 1))

    return step_size


def create_slices(data, window_size):
    step_size = calculate_step_size(len(data), window_size)
    windows = [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)]
    return windows

def calculate_hurst_dfa(time_series,show=False):
    hurst_exponents =[]
    fit_scores =[]
    window_size = 112
    # column_slices = create_slices(time_series,window_size)

    # if column_slices == []:
    #     hurst_exponents.append(0)
    #     fit_scores.append(0)
    # else:
        # for i, window in enumerate(column_slices):
    hurst_value= 0
    window_sizes = calculate_window_sizes(len(time_series))
    hurst_value,r_squared = dfa(time_series,window_sizes,show=True)
    # if(r_squared < 0.95):
    #     hurst_value = 0
    hurst_exponents.append(hurst_value)
    fit_scores.append(r_squared)
    return hurst_exponents,fit_scores

Example DFA Hurst Exponent:

In [ ]:
# # Example Usage:
time_series_data = [-0.0017928385431826186, 0.12523051095819668, 0.15675004725212976, 0.11124722197605776, 0.06990221378029653, 0.05371896659605649, 0.024071357592601234, -0.010131349614161938, 0.027633408068171668, 0.16165649921465566, 0.28542888801566957, 0.262034976554124, 0.08193245613209767, -0.12581938265680126, -0.2279459358019513, -0.19424932138447076, -0.08667456487314125, 0.00875985737537579, 0.03496962384090565, -0.00992013560439811, -0.07475316319179794, -0.0930233134453674, -0.03549503935461722, 0.06357033812676069, 0.1296659269126457, 0.10529157208049339, -0.002489661478363574, -0.12392538400829951, -0.18032232658485622, -0.14637810138174698, -0.06461113435812615, 0.0008813169932818818, 0.02209112573944713, 0.01885805489348043, 0.02213245308107517, 0.036638599583005575, 0.04213377993743712, 0.022868800786330325, -0.013567652207144006, -0.04679865806059885, -0.06756828478953678, -0.08652234093326636, -0.11382241863945324, -0.12937270928903766, -0.0888791592841364, 0.02141192147811009, 0.14294473262281288, 0.1857691116327778, 0.12255890506125997, 0.026547837197958654, 0.0019794775973665784, 0.07945677174202927, 0.1856881807003871, 0.21541148214597003, 0.132594581695328, 0.0054433872294189205, -0.05803717493443957, -0.01743612623419404, 0.05510403231313167, 0.04903735374510066, -0.06278672907669842, -0.1934172864174408, -0.2392237404531362, -0.18718422655059097, -0.10917196457732985, -0.06520036216963121, -0.04022529428235227, 0.014658434425865749, 0.0981797258429603, 0.1472349241018424, 0.11686308386040548, 0.040630565025067227, -0.016853350196586285, -0.045818055028717664, -0.0873657820449646, -0.1442197069749069, -0.14211025372739683, -0.020844694704800725, 0.14673808926437473, 0.1863144199452584, 0.019051726332984395, -0.20451599848581586, -0.2433762574532787, -0.04767778723420615, 0.17225753073183814, 0.19027668087662858, 0.011405962037251282, -0.16950509724571383, -0.19591820826434503, -0.07564626812561454, 0.08312969836720303, 0.19092152796828407, 0.22650506018946695, 0.21073537582966648, 0.15819430964821773, 0.061543056835714614, -0.07056147065145364, -0.17601061799859252, -0.18071110686106803, -0.08611366945813145, 0.025339475364129435, 0.08608378006072419, 0.09986969596065941, 0.08519692520888547, 0.022262862105094372, -0.09898181389345735, -0.22488267238936768, -0.2890684431607644, -0.28771230908674905, -0.25072169855338644, -0.1659652078135314, 0.003097529992504004, 0.23073744925770517, 0.4145474264877708, 0.4567668862640695, 0.338969869813705, 0.13636354370739476, -0.013846343158207519, 0.000570326176883506, 0.14802003565849, 0.24125833559071502, 0.11878331884690728, -0.15621189444674352, -0.34339859809934, -0.28828877326163294, -0.08158075856772491, 0.0614832965208791, 0.02041183282282329, -0.16279738350794423, -0.3716994413668554, -0.503910011142099, -0.4991234108064798, -0.3552966161602029, -0.13645144835321402, 0.07211957502118807, 0.22851890675013548, 0.33556333946066136, 0.39053605738075675, 0.37442099209006413, 0.29478740654995256, 0.20342062179863207, 0.15494686022657084, 0.1622277763055538, 0.19559902338027005, 0.2105971901956893, 0.17572520789753315, 0.09026035395150932, -0.019635077702882395, -0.12958210690497524, -0.23520032100235994, -0.32840095568021077, -0.37138769707066166, -0.3318025061600793, -0.2469258145604436, -0.2063639425257068, -0.2385034162760304, -0.2476005878209344, -0.11663697141142366, 0.13241117608405975, 0.3305591013662113, 0.32949640877027864, 0.14914357244758852, -0.04890416781361836, -0.10812635705217796, 0.0046788235814435265, 0.191676711260044, 0.3114659688326604, 0.2862645597193996, 0.1467326753137744, -0.00958261417722743, -0.09757306380566776, -0.09400384995747946, -0.029151994171915153, 0.04924144794742821, 0.0936488347417316, 0.061855932291594376, -0.05078521685692597, -0.17519240525747637, -0.20782034658358767, -0.12103276321515428, -0.014853468979535722, -0.017102437370092413, -0.126824080658581, -0.1979628810502628, -0.11378093527430146, 0.04749364158422634, 0.08322776382514499, -0.08190980424295076, -0.27252821351916373, -0.24651083350016934, 0.01648461518809675, 0.2762146489779186, 0.30484349711875114, 0.1432726905568514, 0.024328118301811596, 0.07330744287820426, 0.18213497756370736, 0.19926903148955338, 0.14024761976139197, 0.12771346472633177, 0.18469420886372115, 0.20169233312005924, 0.10746806699315523, -0.0316851519058008, -0.1224147577764592, -0.17434529449744332, -0.2563175799280038, -0.3717707594818455, -0.4482604358421939, -0.4313530666636128, -0.3317388431871029, -0.1919984182981693, -0.05629472240547677, 0.0403743993497134, 0.10331206623544145, 0.19004196401522752, 0.3384537427100828, 0.47688153110409476, 0.4576847023502513, 0.21965883657804647, -0.10257392613639324, -0.2810330779047508, -0.2217935488444537, -0.05532225730168795, 0.023956204598837398, -0.02568345101197211, -0.07905644612800879, -0.0265393117558892, 0.09215957815699363, 0.1600140890947295, 0.13686361678134187, 0.08049272751844105, 0.03319728040709229, -0.030570227233554428, -0.11626822018784308, -0.1410963387075328, -0.02597966854469792, 0.16451274390080592, 0.24810318346860166, 0.11952884919729477, -0.11786967917420503, -0.2503447695725205, -0.1695627061166155, 0.03433443547085666, 0.17952841625702895, 0.15930344842973543, 0.008917099077916308, -0.1537019723755309, -0.23501703834578772, -0.2285778306238155, -0.18914880200702575, -0.1590606461018535, -0.13463330490145647, -0.0955036274518545, -0.03504766325864384, 0.04065518497016666, 0.11346666986089336, 0.15117078764147868, 0.1348317585742261, 0.08714221571233999, 0.04564086598925277, 0.01657594611455037, -0.008847684944479798, -0.0016756926821048564, 0.08683172318964218, 0.23325981170890997, 0.31169497036244775, 0.20594497718998117, -0.04862736522222998, -0.27307563912967114, -0.3135581736877329, -0.18448533796664593, -0.028322712759915856, 0.04639723614080599, 0.05372847460023967, 0.06056906520827326, 0.07690548975738984, 0.05907657068685738, 0.0020599998124161147, -0.037876693205869144, -0.030731236033970155]

# H, c, data = compute_Hc(time_series_data)
hurst_value = calculate_hurst_dfa(time_series_data,True)
plt.figure(figsize=(20,6))
plt.plot(time_series_data)
plt.xlabel("Timepoints")
plt.ylabel("Magnitude")
plt.grid(True)
print("Hurst Exponent:", hurst_value)
print("Length of the time series:", len(time_series_data))


Power noise test for evaluating correctness of Hurst evaluation

In [ ]:
def power_law_noise(n, alpha, var=1):
    '''
    Generale power law noise. 
    
    Args:
    -----
      *n* : int
        number of data points
      *alpha* : float
        DFA exponent
      *var* = 1 : float
        variance
    Returns:
    --------
      *x* : numpy.array
        generated noisy data with exponent *alpha*

    Based on:
    N. Jeremy Kasdin, Discrete simulation of power law noise (for
    oscillator stability evaluation)
    '''
    # computing standard deviation from variance
    stdev = np.sqrt(np.abs(var))
    beta = 2*alpha-1
    hfa = np.zeros(2*n)
    hfa[0] = 1
    for i in range(1,n):
        hfa[i] = hfa[i-1] * (0.5*beta + (i-1))/i
    # sample white noise
    wfa = np.hstack((-stdev +2*stdev * np.random.rand(n), np.zeros(n)))
    fh = np.fft.fft(hfa)
    fw = np.fft.fft(wfa)
    fh = fh[1:n+1]
    fw = fw[1:n+1]
    ftot = fh * fw
    # matching the conventions of the Numerical Recipes
    ftot = np.hstack((ftot, np.zeros(n-1)))
    x = np.fft.ifft(ftot)    
    return np.real(x[:n])



n = 128
dfa_alpha = 0.9
time_series_data = power_law_noise(n, dfa_alpha)
hurst_value = calculate_hurst_dfa(time_series_data)
print("Hurst Exponent:", hurst_value)


Visualising the preprocessed data with the following:
    - atlas : no atlas
    - masking : EPI (Echo Planar Imaging)

In [ ]:
fmri_filename = "../output_directory/sub-01/func/sub-01_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_postproc_smooth.nii.gz"

# initialising a masker without atlas
brain_masker = maskers.NiftiMasker(mask_strategy="whole-brain-template")

# Apply masker
time_series_epi = brain_masker.fit_transform(fmri_filename,
                                               confounds=None)

print("Dimension of the fMRI signal (x,y,z,timestamps): ", nibabel.load(fmri_filename).shape)
print("Dimension of the EPI time series (timestamps , voxels): ", time_series_epi.shape)

# Plotting time series of a voxel
voxel = 1

plt.figure(figsize=(20,3))
plt.plot(time_series_epi[:,voxel])
plt.title("Timeseriese of a voxel")
plt.xlabel("Time (s)", fontsize = 10)
plt.ylabel("BOLD signal", fontsize= 10)
plt.grid()
plt.show()

# print(time_series_epi)

report = brain_masker.generate_report()
report

Visualising the preprocessed data with the following:
    - atlas : msdl (Multi Subject Dictionary Learning)
    - masking : atlas + no confounds

In [ ]:
fmri_filename = "Dataset/sub-01/func/sub-01_task-rest_run-1_bold.nii.gz"
# "preprocessed_dataset/sub-025_ses-base_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
data_dir = "../python_scripts/resources/atlas"

#MSDL atlas:
msdl_atlas = datasets.fetch_atlas_msdl(data_dir=data_dir)

msdl_coords = msdl_atlas.region_coords
n_regions = len(msdl_coords)

print(f'MSDL has {n_regions} ROIs, part of the following networks:{np.unique(msdl_atlas.networks)}.')

# initialising a masker

msdl_masker = maskers.NiftiMapsMasker(
    msdl_atlas.maps, resampling_target="data", detrend=True)


# Apply masker
time_series_msdl = msdl_masker.fit_transform(fmri_filename,confounds=None)

print("Dimension of the fMRI signal (x,y,z,timestamps): ", nibabel.load(fmri_filename).shape)
print("Dimension of the msdl time series (timestamps , voxels): ", time_series_msdl.shape)

# Plotting time series of a voxel
voxel = 38

plt.figure(figsize=(20,3))
plt.plot(time_series_msdl[:, voxel], marker = 'o')
plt.title(f'Timeseriese of {msdl_atlas.networks[voxel]} region')
plt.xlabel("Time (s)", fontsize = 10)
plt.ylabel("BOLD signal", fontsize= 10)
plt.grid()
plt.show()

#Hurst component analysis:
hurst_exponents_msdl,hurst_constants_msdl,hurst_data_msdl = calculate_hurst(time_series_msdl)

plt.figure(figsize=(20,3))
plt.plot(hurst_exponents_msdl, marker = 'o')
plt.title('Hurst exponent of every voxels')
plt.xlabel("Voxels", fontsize = 10)
plt.ylabel("Hurst exponent", fontsize= 10)
plt.grid()
plt.show()

report = msdl_masker.generate_report()
report

Obtaining the repeatition time for the fmri signals:

In [ ]:
img_preprocessed = image.load_img('../preprocessed_dataset/sub-025_ses-base_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
img_raw = image.load_img("../Dataset/sub-01/func/sub-01_task-rest_run-1_bold.nii.gz")

tr_raw = img_raw.header.get_zooms()[-1]  # Get the TR_raw
print("Repetition Time (TR_raw):", tr_raw)
print(img_raw.header.get_zooms())

tr_preprocessed = img_preprocessed.header.get_zooms()[-1]  # Get the TR_preprocessed
print("Repetition Time (TR_preprocessed):", tr_preprocessed)
print(img_preprocessed.header.get_zooms())

Function to extract the path of files with specific name structure.

In [ ]:
def get_file_path(main_directory,file_name):
    """
    Get the path of files for different subjects.
    
    Args:
    -----
      *main_directory* : string
        location of the main directory having all the subjects.
      *file_name* : string
        end of the file name which you are looking for including the file type
    Returns:
    --------
      *file_path* : dictionary
        dictionary with keys as subject_numbers and values as respective file path.
    """
    

    # Get a list of subject directories that start with "sub-"
    subject_directories = sorted([d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d)) and d.startswith("sub-")])

    # Initialize lists to store data
    fmri_path = []
    file_path = {}

    # Loop through subject directories
    for subject_dir in subject_directories:
        fmri_path = []
        subject_path = os.path.join(main_directory, subject_dir)
        func_dir = os.path.join(subject_path, "func")

        if os.path.exists(func_dir) and os.path.isdir(func_dir):
            # Assuming fMRI files have a common pattern, such as '*.nii.gz' inside the "func" subdirectory
            fmri_files = [f for f in os.listdir(func_dir) if f.endswith(file_name)]

            # Loop through fMRI files
            for fmri_file in fmri_files:
                fmri_path.append(os.path.join(func_dir, fmri_file))
            file_path[subject_dir]= sorted(np.array(fmri_path))

    return(file_path)

Code to extract number of rest and sleep files of all subjects:

In [ ]:
import os
import pandas as pd

# Function to process a single TSV file
def process_tsv(file_path):
    # Read the TSV file into a DataFrame
    df = pd.read_csv(file_path, sep='\t')

    # Extract subject and run information from the file name
    file_name = os.path.basename(file_path)
    subject = file_name.split('-s')[0]

    # Group by run and calculate max epoch time and NREM states
    grouped_data = df.groupby(['session', 'epoch_start_time_sec']).agg({'30-sec_epoch_sleep_stage': 'max'}).reset_index()
    # Separate sleep runs and rest runs
    sleep_runs = grouped_data[grouped_data['session'].str.contains('sleep_run')]
    rest_runs = grouped_data[grouped_data['session'].str.contains('rest_run')]

    # Calculate max epoch time and associated sleep states for each sleep run
    max_time_and_states_sleep = {}
    for run in sleep_runs['session'].unique():
        run_data = sleep_runs[sleep_runs['session'] == run]
        max_epoch_time = run_data['epoch_start_time_sec'].max()
        sleep_states = run_data.sort_values('epoch_start_time_sec')['30-sec_epoch_sleep_stage'].tolist()
        max_time_and_states_sleep[run] = {'max_epoch_time': max_epoch_time, 'sleep_states': sleep_states}

    # Calculate max epoch time and associated sleep states for each rest run
    max_time_and_states_rest = {}
    for run in rest_runs['session'].unique():
        run_data = rest_runs[rest_runs['session'] == run]
        max_epoch_time = run_data['epoch_start_time_sec'].max()
        sleep_states = run_data.sort_values('epoch_start_time_sec')['30-sec_epoch_sleep_stage'].tolist()
        max_time_and_states_rest[run] = {'max_epoch_time': max_epoch_time, 'sleep_states': sleep_states}

    return subject, max_time_and_states_sleep, max_time_and_states_rest

# Function to process all TSV files in a directory
def process_directory(directory_path):
    source_data = {}

    # Loop through all files in the directory
    for file_name in sorted(os.listdir(directory_path)):
        if file_name.endswith('.tsv'):
            file_path = os.path.join(directory_path, file_name)

            subject, max_time_and_states_sleep, max_time_and_states_rest = process_tsv(file_path)

            # Create or update the dictionary
            if subject not in source_data:
                source_data[subject] = {}
            source_data[subject]['max_time_and_states_sleep'] = max_time_and_states_sleep
            source_data[subject]['max_time_and_states_rest'] = max_time_and_states_rest

    return source_data

def save_to_csv(data, output_file):
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

directory_path = '../Dataset/sourcedata'
result = process_directory(directory_path)

save_to_csv(result, '../Dataset/source_data_summary.csv')


Script to analyse the number of rest and sleep runs vs subjects.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 


def analyze_runs(data):
    # Count the number of rest and sleep runs per user
    rest_runs_count = {}
    sleep_runs_count = {}

    for user, user_data in data.items():
        rest_runs_count[user] = len(user_data.get('max_time_and_states_rest', {}))
        sleep_runs_count[user] = len(user_data.get('max_time_and_states_sleep', {}))

    # Plot bar graphs
    plot_bar_graph(rest_runs_count, 'Number of rest sessions per subject', 'Number of Rest Sessions')
    plot_bar_graph(sleep_runs_count, 'Number of sleep sessions per subject', 'Number of Sleep Sessions')

    # Count the number of subjects with the same number of rest and sleep runs
    count_dict_rest = count_subjects_same_runs(rest_runs_count, 'Rest Runs')
    count_dict_sleep = count_subjects_same_runs(sleep_runs_count, 'Sleep Runs')

    # Plot bar graphs for subjects with the same number of rest and sleep runs
    plot_count_subjects_bar_graph(count_dict_rest, 'Rest Runs')
    plot_count_subjects_bar_graph(count_dict_sleep, 'Sleep Runs')

def plot_bar_graph(data, title, ylabel):
    users = list(data.keys())
    counts = list(data.values())
    plt.figure(figsize=(20,6))
    plt.bar(users, counts, color='skyblue')
    plt.xlabel('Subjects')
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')  
    plt.title(title)
    plt.show()

def count_subjects_same_runs(data, run_type):
    count_dict = {}
    for count in set(data.values()):
        subjects_with_count = [subject for subject, count_subject in data.items() if count_subject == count]
        count_dict[count] = subjects_with_count

    print(f"\nNumber of subjects with the same number of {run_type}:")
    for count, subjects in count_dict.items():
        print(f"{count} {run_type}: {len(subjects)} subjects - {subjects}")

    return count_dict

def plot_count_subjects_bar_graph(count_dict, run_type):
    counts = list(count_dict.keys())
    subjects_count = [len(subjects) for subjects in count_dict.values()]

    x_ticks = np.arange(0,11,1) 
    plt.figure(figsize=(20,6))
    plt.bar(counts, subjects_count, color='skyblue')
    plt.xlabel(f'Number of {run_type}')
    plt.ylabel('Number of Subjects')
    plt.title(f'Subjects with the Same Number of {run_type}')
    plt.xticks(x_ticks)
    plt.show()

analyze_runs(result)


Comparing epochs time of each subjects:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def analyze_runs(data):
    # Extract max epoch time for each rest and sleep run
    max_epoch_time_rest = {}
    max_epoch_time_sleep = {}
    
    for user, user_data in data.items():
        max_epoch_time_rest[user] = [run_data['max_epoch_time'] for run_data in user_data.get('max_time_and_states_rest', {}).values()]
        max_epoch_time_sleep[user] = [run_data['max_epoch_time'] for run_data in user_data.get('max_time_and_states_sleep', {}).values()]
    
    # Plot bar graphs for max epoch time for rest and sleep runs
    plot_max_epoch_time_bar_graph(max_epoch_time_rest, 'Rest Runs', 'Max Epoch Time for Rest Runs')
    plot_max_epoch_time_bar_graph(max_epoch_time_sleep, 'Sleep Runs', 'Max Epoch Time for Sleep Runs')

    # Identify subjects with the same max epoch time for rest and sleep runs
    count_subjects_same_max_time(max_epoch_time_rest, 'Rest Runs')
    count_subjects_same_max_time(max_epoch_time_sleep, 'Sleep Runs')

def plot_max_epoch_time_bar_graph(data, run_type, ylabel):
    users = list(data.keys())
    max_epoch_times = [np.mean(max_times) for max_times in data.values()]
    
    plt.figure(figsize=(20,6))
    plt.bar(users, max_epoch_times, color='skyblue')
    plt.xlabel('Users')
    plt.ylabel(ylabel)
    plt.xticks(rotation=90)
    plt.title(f'{ylabel} for {run_type}')
    plt.show()

def count_subjects_same_max_time(data, run_type):
    count_dict = {}
    for max_time in set(np.concatenate(list(data.values()))):
        subjects_with_max_time = [user for user, max_times in data.items() if max_time in max_times]
        count_dict[max_time] = subjects_with_max_time

    print(f"\nSubjects with the same max epoch time for {run_type}:")
    for max_time, subjects in count_dict.items():
        print(f"Max Epoch Time {run_type}: {max_time}, Total Subjects: {len(subjects)}")

# Assuming you already have the 'result' dictionary
analyze_runs(result)


Code for Framewise Displacement evaluation:
    The code extracts all the "desc-confounds_timeseries.tsv" file locations per subjects
        if it is less than threshold value (0.7), if not then it stores the value and the timestamp(which is the row number + 1)
        Calculates the avg of FD values and check if that is less than the threshold as well.
    The script finally gives a csv file with the Quality check considering Framewise displacement

In [ ]:
confounds_path = get_file_path("../output_directory", "desc-confounds_timeseries.tsv")

def quality_check_fd(confounds_files, avg_threshold,voxel_vol_threshold):
    """
    The code runs through the framwise displacement values per runs of each subject, extracts FD values and checks:
        if it is less than threshold_avg,threshold_voxel value (0.7), if not then it stores the value and the timestamp(which is the row number + 1)
        Calculates the avg of FD values and check if that is less than the threshold as well.
    Args:
    -----
      *confounds_files* : dictionary
        dictionary with keys as subject_numbers and values as respective file path.

      *avg_threshold* : float
        threshold value of the average of the framewise displacement values

      *voxel_vol_threshold* : float
        threshold value of framewise displacement per voxel volume (Individual Repeatition time)

    Returns:
    --------
      *quality_check_results* : dictionary
        dictionary of the quality check of every run per subject based on framewise displacement
    """ 

    quality_check_results = []
    for subject, file_paths in confounds_files.items():
        for file_path in file_paths:
            subject_data = {"Subject Number": subject, "File Name": os.path.basename(file_path),
                            "FD values(2.3mm - 4.3mm)": [],"Exceeding Threshold(2.3mm - 4.3mm)": 0,"Exceeding Timepoints(2.3mm - 4.3mm)": [], 
                            "FD values(4.3mm - 6.3mm)": [],"Exceeding Threshold(4.3mm - 6.3mm)": 0,"Exceeding Timepoints(4.3mm - 6.3mm)": [],
                            "FD values(>6.3mm)": [],"Exceeding Threshold(>6.3mm)": 0,"Exceeding Timepoints(>6.3mm)": [],
                            "Average FD": 0.0, "Avg > threshold": ""}

            df = pd.read_csv(file_path, delimiter='\t')

            fd_values = df['framewise_displacement'].tolist()
            subject_data["Average FD"] = sum(fd_values[1:]) / len(fd_values[1:])
            if subject_data["Average FD"]>avg_threshold:
                subject_data["Avg > threshold"] = "YES"
            else:
                subject_data["Avg > threshold"] = "NO"

            exceeding_timepoints = [i+1 for i, fd in enumerate(fd_values) if voxel_vol_threshold <fd < (voxel_vol_threshold+2)]
            subject_data["Exceeding Timepoints(2.3mm - 4.3mm)"].extend(exceeding_timepoints)
            subject_data["Exceeding Threshold(2.3mm - 4.3mm)"] += len(exceeding_timepoints)
            fd_above_threshold = [fd for fd in fd_values if voxel_vol_threshold <fd < (voxel_vol_threshold+2)]
            subject_data["FD values(2.3mm - 4.3mm)"].extend(fd_above_threshold)

            exceeding_timepoints = [i+1 for i, fd in enumerate(fd_values) if (voxel_vol_threshold+2) <fd < (voxel_vol_threshold+4)]
            subject_data["Exceeding Timepoints(4.3mm - 6.3mm)"].extend(exceeding_timepoints)
            subject_data["Exceeding Threshold(4.3mm - 6.3mm)"] += len(exceeding_timepoints)
            fd_above_threshold = [fd for fd in fd_values if (voxel_vol_threshold+2) <fd < (voxel_vol_threshold+4)]
            subject_data["FD values(4.3mm - 6.3mm)"].extend(fd_above_threshold)

            exceeding_timepoints = [i+1 for i, fd in enumerate(fd_values) if fd > (voxel_vol_threshold+4)]
            subject_data["Exceeding Timepoints(>6.3mm)"].extend(exceeding_timepoints)
            subject_data["Exceeding Threshold(>6.3mm)"] += len(exceeding_timepoints)
            fd_above_threshold = [fd for fd in fd_values if fd > (voxel_vol_threshold+4)]
            subject_data["FD values(>6.3mm)"].extend(fd_above_threshold)

            quality_check_results.append(subject_data.copy())  # Use copy to create a new dictionary for each subject

    return quality_check_results


avg_threshold_value = 0.5
voxel_vol_threshold_value =2.3
output_csv_file = "../output_directory/qc_fd_report_3.csv"

qc_fd_results = quality_check_fd(confounds_path, avg_threshold_value,voxel_vol_threshold_value)

df = pd.DataFrame(qc_fd_results)
df.to_csv(output_csv_file, index=False)


Code to extract the run number from the file name

In [ ]:
def extract_task_run_from_filename(filename):
    pattern = r'task-(rest|sleep)_run-\d+'
    match = re.search(pattern, filename)
    return match.group() if match else None

 Code for visualising the average framewise displacement per 30 second epoch wrt NREM state:
    First extracts all the "desc-confounds_timeseries.tsv" file locations per subjects
    Make a csv file with average FD values for 30 seconds epoch for every runs per subject
    Add the sleep scores based on the runs per 30 seconds per subjects.

In [ ]:
%matplotlib inline
import ptitprince as plt

def epoch_fd_avg(confounds_files, window_size_seconds=30):
    """
    The code runs through the framwise displacement values per runs of each subject, extracts FD values and stores Average of framewise displacement for a window of 30 seconds each
    
    Args:
    -----
      *confounds_files* : dictionary
        dictionary with keys as subject_numbers and values as respective file path.

      *window_size_seconds* : int
        window size in seconds: default of 30 seconds epochs based on our dataset
    
    Returns:
    --------
      *epoch_fd_results* : dictionary
        dictionary of averaged framewise displacement per 30 second epochs
    """ 

    epoch_fd_results = []

    for subject, file_paths in confounds_files.items():
        for file_path in file_paths:
            task_run = extract_task_run_from_filename(os.path.basename(file_path))
            df = pd.read_csv(file_path, delimiter='\t')

            fd_values = df['framewise_displacement'].tolist()

            window_size = int(window_size_seconds / 2.1)

            num_windows = len(fd_values) // window_size

            for i in range(num_windows):
                start_idx = i * window_size
                end_idx = start_idx + window_size
                window_fd_values = fd_values[start_idx:end_idx]
                window_fd_values = [fd for fd in window_fd_values if not pd.isna(fd)]

                window_data = {
                    "Subject Number": subject,
                    "File Name": task_run,
                    "Window Start": start_idx * 2.1,  
                    "Window End": end_idx * 2.1,
                    "Average FD": sum(window_fd_values) / len(window_fd_values),
                    "Exceeding Threshold": sum(1 for fd in window_fd_values if fd > 0.7)
                }

                epoch_fd_results.append(window_data)
    return epoch_fd_results


output_csv_file = "../output_directory/epoch_fd_report.csv"
confounds_path = get_file_path("../output_directory", "desc-confounds_timeseries.tsv")

confounds_path=filtered_data = {key: value for key, value in confounds_path.items() if key != 'sub-04'}

epoch_fd_results = epoch_fd_avg(confounds_path,window_size_seconds=30)
df = pd.DataFrame(epoch_fd_results)
df.to_csv(output_csv_file, index=False)

Code for converting all sleep score tsv files into one CSV

In [ ]:
def extract_subject_number(file_name):
    # Assuming the subject number is extracted from the file name (modify accordingly)
    # This is just a simple example, adjust as per your file naming convention
    return file_name[0:6]

def create_combined_csv(tsv_folder_path, output_csv_path):
    # Initialize an empty DataFrame to store the combined data
    df_combined = pd.DataFrame(columns=['Subject Number'])

    # Iterate through each TSV file in the folder
    for tsv_file in sorted(os.listdir(tsv_folder_path)):
        if tsv_file.endswith('.tsv'):
            tsv_file_path = os.path.join(tsv_folder_path, tsv_file)

            # Read the TSV file
            df_tsv = pd.read_csv(tsv_file_path, delimiter='\t')

            # Extract subject number from the file name
            subject_number = extract_subject_number(tsv_file)

            # Add 'Subject Number' column to the TSV data
            df_tsv['Subject Number'] = subject_number

            # Select and rename columns
            df_tsv = df_tsv[['Subject Number', 'session', 'epoch_start_time_sec', '30-sec_epoch_sleep_stage']]
        

            # Append the data to the combined DataFrame
            df_combined = pd.concat([df_combined, df_tsv],ignore_index=True)

    # Save the combined DataFrame to a CSV file
    df_combined.to_csv(output_csv_path, sep=',', index=False)

# Example usage:
tsv_folder_path = '../Dataset/sourcedata'
output_csv_path = '../Dataset/sourcedata/all_nrem.csv'
create_combined_csv(tsv_folder_path, output_csv_path)


Visualising the Average FD wrt sleep score:
        Plot the raincloud plot for visualising FD vs Sleep states

In [ ]:
%matplotlib inline
import ptitprince as pt


df = pd.read_csv("../output_directory/epoch_fd_report.csv")

f,ax=plt.subplots(figsize=(10,5))
pt.RainCloud(x="30-sec_epoch_sleep_stage", y="Average FD", data=df, width_viol=0.5, width_box=0.4, orient='h',order= ['W','1','2'])

# Set additional properties or customization if needed
ax.set_title("Framewise Displacement vs Sleep Stages")
ax.set_ylabel("Sleep Stages")
ax.set_xlabel("Maximum Framewise Displacement(mm)")

# Show the plot
plt.show()

Plotting Subject vs FD:

In [ ]:
%matplotlib inline
import ptitprince as pt


df = pd.read_csv("../output_directory/qc_fd_report.csv")

fd_per_subject = df[["Subject Number","Average FD"]].groupby("Subject Number").mean()


subject_numbers = fd_per_subject.index.astype(str)  
average_fds = fd_per_subject["Average FD"]

# Create a list of colors based on the value of FD
colors = ['red' if fd > 0.4 else 'green' for fd in average_fds]

# Create the bar graph
plt.figure(figsize=(10, 6))
plt.bar(subject_numbers, average_fds, color=colors)

# Add labels
plt.xlabel("Subjects")
plt.ylabel("Average Framewise Displacement (mm)")
plt.title("Average Framewise Displacement by Subject")
plt.xticks(rotation=45, ha='right')  

plt.tight_layout() 
plt.show()

In [ ]:
%matplotlib inline
import ptitprince as pt


df = pd.read_csv('../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_roi.csv')

fd_per_subject = df[["sleep_stage","Total datapoints"]].groupby("sleep_stage").mean()


subject_numbers = fd_per_subject.index.astype(str)  
average_fds = fd_per_subject["Total datapoints"].drop("3")/28
desired_order = ["W", "1", "2"]

average_fds = average_fds.reindex(desired_order)
# Create the bar graph
plt.figure(figsize=(10, 6))
plt.bar(["Wake", "NREM1","NREM2"], average_fds, color='green')

# Add labels
plt.xlabel("Sleep Scores")
plt.ylabel("Data length (min)")
plt.title("Average data length per sleep score") 

plt.tight_layout() 
plt.show()

Postprocessing script:
    

In [ ]:
from postprocessing import postproc_script 

img_files = get_file_path("../output_directory", "desc-preproc_bold.nii.gz")
img__mask_files = get_file_path("../output_directory", "_desc-brain_mask.nii.gz")
conf_files = get_file_path("../output_directory", "desc-confounds_timeseries.tsv")

selected_keys = ['sub-31','sub-32','sub-33']

# Creating a new dictionary with only the selected keys
img_files = {key: img_files[key] for key in selected_keys}
img__mask_files = {key: img__mask_files[key] for key in selected_keys}
conf_files = {key: conf_files[key] for key in selected_keys}

postproc_csv_files = postproc_script(img_files, img__mask_files, conf_files)

print(postproc_csv_files)

Visualising the postprocessed data with the following:
    - atlas : Harvard Oxford Atlas: 2mm
    - masking : atlas + no confounds
    -storing time series of all regions in a csv

In [ ]:
def get_ho_region_ts_per_epoch(fmri_path,atlas_dir,window_size_seconds=30,display_report=False):
    """
    The code applies Harvard Oxford Atlas to the fMRI file and extracts time series of each of the 48 regions in windows of 30 seconds.
    
    Args:
    -----
      *fmri_path* : string
        path to the fmri file

      *atlas_dir* : string
        path to the Harvard Oxford atlas directory

      *window_size_seconds* : int
        window size in seconds

      *display_report* : boolean
        parameter to decide to display the report of the atlas or not
      
    Returns:
    --------
      *region_ts* : 2D array
        a 2D array having 
          columns: representing each of the 48 region in the HO atlas 
          rows: representing the time series of window_size_seconds
    """ 

    # Fetch cortical atlas
    cortical_atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', data_dir=atlas_dir)
    cortical_maps = cortical_atlas.maps
    cortical_labels = cortical_atlas.labels
    cortical_masker = maskers.NiftiLabelsMasker(labels_img=cortical_maps, labels= cortical_labels)


    # Fetch subcortical atlas
    subcortical_atlas = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm', data_dir=atlas_dir)
    subcortical_maps = subcortical_atlas.maps
    subcortical_labels = subcortical_atlas.labels
    subcortical_masker = maskers.NiftiLabelsMasker(labels_img=subcortical_maps, labels= subcortical_labels)
    
    
    combined_labels = cortical_labels[1:] + subcortical_labels[1:]  # Skip 'Background' label from subcortical labels
    n_regions = len(combined_labels)

    # Apply masker
    time_series_cortical = cortical_masker.fit_transform(fmri_path, confounds=None)
    time_series_subcortical = subcortical_masker.fit_transform(fmri_path, confounds=None)
    time_series_combined = np.concatenate((time_series_cortical,time_series_subcortical),axis=1)

    # Display report if needed
    if display_report:
        print("Dimension of the fMRI signal (x,y,z,timestamps): ", nibabel.load(fmri_path).shape)
        print("Dimension of the combined time series (timestamps, voxels): ", time_series_combined.shape)
        print(f'Combined atlas has {n_regions} ROIs.')
        print(combined_labels)

    window_size = int(window_size_seconds / 2.1)  # The normal window size
    first_window_correction = 5  # TRs to remove for the first epoch

    # Adjusted to account for the shortened first window
    adjusted_length = len(time_series_combined[:, 0]) - first_window_correction
    num_windows = 2 + ((adjusted_length - window_size) // window_size)
    n_regions = time_series_combined.shape[1]
    region_ts = np.empty((num_windows, n_regions), dtype=object)
    region_index = 0

    for region in zip(*time_series_combined):
        # Special handling for the first window
        start_idx = 0
        end_idx = window_size - first_window_correction
        region_ts[0, region_index] = region[start_idx:end_idx]

        # Process subsequent windows
        for i in range(1, num_windows):
            start_idx = (i * window_size) - first_window_correction
            end_idx = start_idx + window_size
            region_ts[i, region_index] = region[start_idx:end_idx]

        region_index += 1

    # Construct DataFrame with appropriate column names
    columns = [f'{label}' for i, label in enumerate(combined_labels, start=1)]
    region_ts_df = pd.DataFrame(region_ts, columns=columns)
    return region_ts_df


atlas_dir = "../python_scripts/resources/atlas"
fmri_files = get_file_path("../output_directory", "postproc_smooth.nii.gz")
# selected_keys = ['sub-02']

# # # Creating a new dictionary with only the selected keys
# fmri_files = {key: fmri_files[key] for key in selected_keys}

ho_ts_dfs = []
for subject, fmri_file in fmri_files.items():
        for file_path in fmri_file:
          region_ts_df = get_ho_region_ts_per_epoch(file_path, atlas_dir,window_size_seconds=30,display_report=False)
          region_ts_df.insert(0, 'run', extract_task_run_from_filename(os.path.basename(file_path)))
          region_ts_df.insert(0, 'subject', subject)
          ho_ts_dfs.append(region_ts_df)
          print(f"{extract_task_run_from_filename(os.path.basename(file_path))} of {subject} successfully completed")

result_df = pd.concat(ho_ts_dfs, ignore_index=True)

result_df.to_csv('../output_directory/evaluation_dataset/harvard_atlas_dataset/ho_atlas_dataset_corrected_filtered.csv', index=False, header=True)


Script helping to get a csv format of all the data in a structure:

In [ ]:
def read_csv(file_path):
    return pd.read_csv(file_path)

# Reading the uploaded CSV file
df = read_csv('../output_directory/evaluation_dataset/harvard_atlas_dataset/ho_atlas_dataset_corrected_filtered.csv')

def process_data(df):
    # Renaming columns for clarity
    df.columns.values[0:3] = ['subject', 'run', 'sleep_stage']

    # Identify consecutive series and assign group IDs
    consecutive_series = (df['subject'] != df['subject'].shift(1)) | \
                         (df['run'] != df['run'].shift(1)) | \
                         (df['sleep_stage'] != df['sleep_stage'].shift(1))
    group_id = consecutive_series.cumsum()
    
    # Group by 'subject', 'run', 'sleep_stage', and group_id
    grouped = df.groupby([df.columns[0], df.columns[1], df.columns[2], group_id],sort=False)
    
    aggregation = {col: lambda x: list(x) for col in df.columns[3:]}
    processed_data = grouped.agg(aggregation).reset_index()

    processed_data.insert(4, 'Total datapoints', processed_data.iloc[:,4].apply(lambda x: len(x)))
    # processed_data['Total datapoints'] = processed_data.apply(lambda row: 9 + (len(row.iloc[4]) - 1) * 14, axis=1)
    processed_data = processed_data.drop('level_3', axis=1)
    return processed_data

processed_df = process_data(df)

def write_to_csv(df, file_path):
    df.to_csv(file_path, index=False)

output_file_path = '../output_directory/evaluation_dataset/harvard_atlas_dataset/eval_dataset_corrected_filtered.csv'
write_to_csv(processed_df, output_file_path)

Hurst Value Evaluation of the final csv dataset:

In [ ]:
import pandas as pd
import re

def read_csv(file_path):
    return pd.read_csv(file_path)

# Convert string representation of list of tuples to a list of floats
def convert_string_to_list(data_string):
    data_string = data_string[1:-1]
    tuples = re.findall(r'\((.*?)\)', data_string)
    list_of_floats = [float(num) for t in tuples for num in t.split(', ')]
    return list_of_floats

# Reading the uploaded CSV file
df = read_csv('../output_directory/evaluation_dataset/harvard_atlas_dataset/eval_dataset_corrected_filtered.csv')


fit_scores={}
region_columns = df.columns[4:]


for region in region_columns:
    fit_scores[region] = df.apply(lambda row: calculate_hurst_dfa(convert_string_to_list(row[region]))[1], axis=1).values
    df[region] = df.apply(lambda row: calculate_hurst_dfa(convert_string_to_list(row[region]))[0], axis=1)
    print(f"{region} completed")


#Fit score calculation:
# fit_scores= {key: [item for sublist in value for item in sublist] for key, value in fit_scores.items()}

# Save the updated DataFrame to a new CSV file
df.to_csv('../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected_filtered.csv', index=False)



In [ ]:
import pandas as pd
import ast
import numpy as np

def safe_eval(val):
    try:
        # Evaluate the string value as a list
        evaluated = ast.literal_eval(val)
        # Check if the evaluated list contains only NaN
        if all(np.isnan(item) if isinstance(item, float) else False for item in evaluated):
            return [0]  # Return list with a single NaN
        return evaluated
    except:
        # Return a list with a single NaN if the value can't be evaluated as a list
        return [0]

# Read the CSV file
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected_filtered.csv'
df = pd.read_csv(file_path)

# Identify Hurst columns
hurst_columns = df.columns[4:]

# Expand each row into multiple rows
expanded_rows = []
for _, row in df.iterrows():
    # Convert the string representations to lists (handling NaN values)
    list_values = [safe_eval(row[col]) for col in hurst_columns]

    # Find the length of the arrays (assuming all arrays are of the same length)
    length = max(len(l) for l in list_values)

    # Create a new row for each element in the arrays
    for i in range(length):
        new_row = row.to_dict()
        for idx, col in enumerate(hurst_columns):
            new_row[col] = list_values[idx][i] if i < len(list_values[idx]) else 0
        expanded_rows.append(new_row)
    

# Create a new DataFrame with the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

expanded_df = expanded_df[expanded_df['Total datapoints'] >= 8]
state_list= expanded_df["run"].apply(lambda x: 'rest' if 'rest' in x else 'sleep')
expanded_df.insert(2, 'state', state_list)
expanded_df['sleep_stage'] = expanded_df['sleep_stage'].apply(lambda x: "W" if (x.split()[0] == 'W') else x.split()[0])


# Save the new DataFrame to a CSV file
expanded_df.to_csv('../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected_filtered.csv', index=False)


Slope Fit Analysis

In [ ]:
mean_slope_fit = {}
for key, values in fit_scores.items():
    non_zero_values = [value for value in values if value != 0]
    mean_slope_fit[key] = np.mean(non_zero_values)

keys = list(mean_slope_fit.keys())
values = list(mean_slope_fit.values())
threshold_value_y =0.95

plt.figure(figsize=(20, 6))
plt.bar(keys, values, color='skyblue')
plt.axhline(y=threshold_value_y, color='r', linestyle='--', label=f'Threshold Fit score: {threshold_value_y}')
plt.xlabel('Regions')
plt.ylabel('Mean of fit score')
plt.title('Mean of fit score for Each Region')
plt.xticks(rotation=90)
plt.legend()
plt.show()

Variance Evaluation:

In [ ]:
import pandas as pd
import re

def read_csv(file_path):
    return pd.read_csv(file_path)

# Convert string representation of list of tuples to a list of floats
def convert_string_to_list(data_string):
    data_string = data_string[1:-1]
    tuples = re.findall(r'\((.*?)\)', data_string)
    list_of_floats = [float(num) for t in tuples for num in t.split(', ')]
    return list_of_floats


def calculate_variance(data):
    # Calculate the mean of the data
    mean = sum(data) / len(data)
    
    # Calculate the squared differences from the mean
    squared_diff = [(x - mean) ** 2 for x in data]
    
    # Calculate the variance as the mean of the squared differences
    variance = sum(squared_diff) / len(data)
    
    return variance

# Reading the uploaded CSV file
df = read_csv('../output_directory/evaluation_dataset/eval_dataset.csv')


fit_scores={}
region_columns = df.columns[4:]


for region in region_columns:
    df[region] = df.apply(lambda row: calculate_variance(convert_string_to_list(row[region])), axis=1)
    print(f"{region} completed")

df = df[df['Total datapoints'] >= 112]
state_list= df["run"].apply(lambda x: 'rest' if 'rest' in x else 'sleep')
df.insert(2, 'state', state_list)
df['sleep_stage'] = df['sleep_stage'].apply(lambda x: "W" if (x.split()[0] == 'W') else x.split()[0])

# Save the updated DataFrame to a new CSV file
df.to_csv('../output_directory/evaluation_dataset/variance_dataset.csv', index=False)



Visualising Scripts:

In [ ]:
import nibabel as nib
from nilearn import datasets, plotting
from matplotlib.colors import ListedColormap

def calculate_mean(file_path, parameter_index):
    
    df = pd.read_csv(file_path)

    columns_to_consider = [df.columns[parameter_index]] + list(df.columns[5:])

    # Filter out zero values
    non_zero_df = df[df[columns_to_consider] != 0]

    # Group by parameter and calculate the mean for each region
    mean_non_zero_hurst_exponents = non_zero_df.groupby(df.columns[parameter_index], as_index=False).mean()

    return mean_non_zero_hurst_exponents

def create_hurst_nifti_cortical(mean_hurst_values):
    # Fetch cortical atlas
    atlas_dir = "../python_scripts/resources/atlas"
    cortical_atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', data_dir=atlas_dir)
    cortical_labels = cortical_atlas.labels

    # Load the atlas image and get the data
    atlas_img = nib.load(cortical_atlas.filename)
    atlas_data = atlas_img.get_fdata()

    hurst_data = np.zeros_like(atlas_data)


    for i, region in enumerate(cortical_labels[1:]):  # Skip the background label
        # Set all voxels with the label i+1 to the mean Hurst value
        hurst_data[atlas_data == i+1] = mean_hurst_values[region]
    hurst_img_cortical = nib.Nifti1Image(hurst_data, atlas_img.affine)
    return hurst_img_cortical


def create_hurst_nifti_subcortical(mean_hurst_values):
    # Fetch subcortical atlas
    atlas_dir = "../python_scripts/resources/atlas"
    subcortical_atlas = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm', data_dir=atlas_dir)
    subcortical_labels = subcortical_atlas.labels

    # Load the atlas image and get the data
    atlas_img = nib.load(subcortical_atlas.filename)
    atlas_data = atlas_img.get_fdata()

    hurst_data = np.zeros_like(atlas_data)


    for i, region in enumerate(subcortical_labels[1:]):  # Skip the background label
        # Set all voxels with the label i+1 to the mean Hurst value
        hurst_data[atlas_data == i+1] = mean_hurst_values[region]
    hurst_img_subcortical = nib.Nifti1Image(hurst_data, atlas_img.affine)
    
    return hurst_img_subcortical


def plot_brain_regions_hurst(group_parameter, hurst_img_cortical, hurst_img_subcortical):
    fig, axes = plt.subplots(nrows=len(hurst_img_cortical), ncols=1, figsize=(10, 10))
    fig.text(0.5, 1, f'Hurst Exponent', va='center', ha='center', rotation='horizontal', fontsize=12)
    plt.tight_layout()

    # Define the color map
    cmap = plotting.cm.bwr_r

    # Plot for cortical region
    for i, (stage, image) in enumerate(hurst_img_cortical.items()):
        data_cortical = image.get_fdata()

        data_subcortical = hurst_img_subcortical[stage].get_fdata()

        display = plotting.plot_stat_map(
            image,
            title=group_parameter[i],
            display_mode='ortho',
            colorbar=False,
            axes=axes[i],
            annotate=False, draw_cross=False,
            figure=fig,
            symmetric_cbar=False,
            cmap=cmap,
        )
        display.add_overlay(hurst_img_subcortical[stage], cmap=cmap,vmax=1, vmin=0.70, colorbar=True)

    plt.show()


Code for visualising hurst exponent vs Sleep score

In [ ]:
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected.csv'
mean_hurst_per_sleep_stage = calculate_mean(file_path, 3)
sleep_stage_dict_cortical = {}
sleep_stage_dict_subcortical = {}

# Iterate over each sleep stage and create a plot
for _, row in mean_hurst_per_sleep_stage.iterrows():
    sleep_stage = row['sleep_stage']
    sleep_stage_dict_cortical[sleep_stage] = create_hurst_nifti_cortical(row)
    sleep_stage_dict_subcortical[sleep_stage] = create_hurst_nifti_subcortical(row)

sleep_stage_dict_cortical = {'W': sleep_stage_dict_cortical['W'], '1': sleep_stage_dict_cortical['1'], '2': sleep_stage_dict_cortical['2']}
sleep_stage_dict_subcortical = {'W': sleep_stage_dict_subcortical['W'], '1': sleep_stage_dict_subcortical['1'], '2': sleep_stage_dict_subcortical['2']}

plot_brain_regions_hurst(["Wake","NREM 1","NREM 2"], sleep_stage_dict_cortical,sleep_stage_dict_subcortical)


In [ ]:
file_path = '../output_directory/evaluation_dataset/dfa_hurst_dataset.csv'
df = pd.read_csv(file_path)

# Create a new column to group sleep stages as 'W' and '1/2'
df['sleep_stage'] = df['sleep_stage'].replace({'W': 'W', '1': '1/2', '2': '1/2','3':'1/2'})

# Define the columns to consider for grouping and calculating mean
columns_to_consider = [df.columns[3]] + list(df.columns[5:])


# Filter out zero values
non_zero_df = df[df[columns_to_consider] != 0]

# Group by 'sleep_stage_grouped' and calculate the mean for each group
mean_hurst_per_state = non_zero_df.groupby([df.columns[3]], as_index=False).mean()

# print(mean_hurst_per_state)

state_dict_cortical = {}
state_dict_subcortical = {}

# Iterate over each sleep stage and create a plot
for _, row in mean_hurst_per_state.iterrows():
    state = row['sleep_stage']
    state_dict_cortical[state] = create_hurst_nifti_cortical(row)
    state_dict_subcortical[state] = create_hurst_nifti_subcortical(row)

state_dict_cortical = {'W': state_dict_cortical['W'], '1/2': state_dict_cortical['1/2']}
state_dict_subcortical = {'W': state_dict_subcortical['W'], '1/2': state_dict_subcortical['1/2']}

plot_brain_regions_hurst(["Rest","Sleep"], state_dict_cortical,state_dict_subcortical)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


file_path = '../output_directory/evaluation_dataset/dfa_hurst_dataset.csv'

df = pd.read_csv(file_path)

columns_to_consider = [df.columns[1],df.columns[3]] + list(df.columns[5:])

# Group by parameter and calculate the mean for each region
mean_hurst_per_time = df[columns_to_consider].groupby([df.columns[1],df.columns[3]],as_index=False).mean()

time_dict_cortical = {}
time_dict_subcortical = {}

# Iterate over each sleep stage and create a plot
for _, row in mean_hurst_per_time.iterrows():
    run = row['run'] + "_" + row['sleep_stage'] 
    time_dict_cortical[run] = create_hurst_nifti_cortical(row)
    time_dict_subcortical[run] = create_hurst_nifti_subcortical(row)


time_dict_cortical_t1 = {'W': time_dict_cortical['task-rest_run-1_W'], '1': time_dict_cortical['task-rest_run-1_1'], '2': time_dict_cortical['task-rest_run-1_2']}
time_dict_subcortical_t1 = {'W': time_dict_subcortical['task-rest_run-1_W'], '1': time_dict_subcortical['task-rest_run-1_1'], '2': time_dict_subcortical['task-rest_run-1_2']}

time_dict_cortical_t2 = {'W': time_dict_cortical['task-rest_run-2_W'], '1': time_dict_cortical['task-rest_run-2_1'], '2': time_dict_cortical['task-rest_run-2_2']}
time_dict_subcortical_t2 = {'W': time_dict_subcortical['task-rest_run-2_W'], '1': time_dict_subcortical['task-rest_run-2_1'], '2': time_dict_subcortical['task-rest_run-2_2']}

time_dict_cortical_t3 = {'W': time_dict_cortical['task-sleep_run-1_W'], '1': time_dict_cortical['task-sleep_run-1_1'], '2': time_dict_cortical['task-sleep_run-1_2']}
time_dict_subcortical_t3 = {'W': time_dict_subcortical['task-sleep_run-1_W'], '1': time_dict_subcortical['task-sleep_run-1_1'], '2': time_dict_subcortical['task-sleep_run-1_2']}

time_dict_cortical_t4 = {'W': time_dict_cortical['task-sleep_run-2_W'], '1': time_dict_cortical['task-sleep_run-2_1'], '2': time_dict_cortical['task-sleep_run-2_2']}
time_dict_subcortical_t4 = {'W': time_dict_subcortical['task-sleep_run-2_W'], '1': time_dict_subcortical['task-sleep_run-2_1'], '2': time_dict_subcortical['task-sleep_run-2_2']}

time_dict_cortical_t5 = {'W': time_dict_cortical['task-sleep_run-3_W'], '1': time_dict_cortical['task-sleep_run-3_1'], '2': time_dict_cortical['task-sleep_run-3_2']}
time_dict_subcortical_t5 = {'W': time_dict_subcortical['task-sleep_run-3_W'], '1': time_dict_subcortical['task-sleep_run-3_1'], '2': time_dict_subcortical['task-sleep_run-3_2']}

time_dict_cortical_t6 = {'W': time_dict_cortical['task-sleep_run-4_W'], '1': time_dict_cortical['task-sleep_run-4_1'], '2': time_dict_cortical['task-sleep_run-4_2']}
time_dict_subcortical_t6 = {'W': time_dict_subcortical['task-sleep_run-4_W'], '1': time_dict_subcortical['task-sleep_run-4_1'], '2': time_dict_subcortical['task-sleep_run-4_2']}

time_dict_cortical_t7 = {'W': time_dict_cortical['task-sleep_run-5_W'], '1': time_dict_cortical['task-sleep_run-5_1'], '2': time_dict_cortical['task-sleep_run-5_2']}
time_dict_subcortical_t7 = {'W': time_dict_subcortical['task-sleep_run-5_W'], '1': time_dict_subcortical['task-sleep_run-5_1'], '2': time_dict_subcortical['task-sleep_run-5_2']}

time_dict_cortical_t8 = {'W': time_dict_cortical['task-sleep_run-6_W'], '1': time_dict_cortical['task-sleep_run-6_1'], '2': time_dict_cortical['task-sleep_run-6_2']}
time_dict_subcortical_t8 = {'W': time_dict_subcortical['task-sleep_run-6_W'], '1': time_dict_subcortical['task-sleep_run-6_1'], '2': time_dict_subcortical['task-sleep_run-6_2']}

time_dict_cortical_t9 = {'W': time_dict_cortical['task-sleep_run-7_W'], '1': time_dict_cortical['task-sleep_run-7_1'], '2': time_dict_cortical['task-sleep_run-7_2']}
time_dict_subcortical_t9 = {'W': time_dict_subcortical['task-sleep_run-7_W'], '1': time_dict_subcortical['task-sleep_run-7_1'], '2': time_dict_subcortical['task-sleep_run-7_2']}


time_dict_cortical_wake = {'W_1': time_dict_cortical['task-rest_run-1_W'], 'W_2': time_dict_cortical['task-rest_run-2_W'], 'W_3': time_dict_cortical['task-sleep_run-1_W'],'W_4': time_dict_cortical['task-sleep_run-2_W'],'W_5': time_dict_cortical['task-sleep_run-3_W'],'W_6': time_dict_cortical['task-sleep_run-4_W'],'W_7': time_dict_cortical['task-sleep_run-5_W'],'W_8': time_dict_cortical['task-sleep_run-6_W'],'W_9': time_dict_cortical['task-sleep_run-7_W']}
time_dict_cortical_1 = {'1_1': time_dict_cortical['task-rest_run-1_1'], '1_2': time_dict_cortical['task-rest_run-2_1'], '1_3': time_dict_cortical['task-sleep_run-1_1'],'1_4': time_dict_cortical['task-sleep_run-2_1'],'1_5': time_dict_cortical['task-sleep_run-3_1'],'1_6': time_dict_cortical['task-sleep_run-4_1'],'1_7': time_dict_cortical['task-sleep_run-5_1'],'1_8': time_dict_cortical['task-sleep_run-6_1'],'1_9': time_dict_cortical['task-sleep_run-7_1']}
time_dict_cortical_2 = {'2_1': time_dict_cortical['task-rest_run-1_2'], '2_2': time_dict_cortical['task-rest_run-2_2'], '2_3': time_dict_cortical['task-sleep_run-1_2'],'2_4': time_dict_cortical['task-sleep_run-2_2'],'2_5': time_dict_cortical['task-sleep_run-3_2'],'2_6': time_dict_cortical['task-sleep_run-4_2'],'2_7': time_dict_cortical['task-sleep_run-5_2'],'2_8': time_dict_cortical['task-sleep_run-6_2'],'2_9': time_dict_cortical['task-sleep_run-7_2']}

time_dict_subcortical_wake = {'W_1': time_dict_subcortical['task-rest_run-1_W'], 'W_2': time_dict_subcortical['task-rest_run-2_W'], 'W_3': time_dict_subcortical['task-sleep_run-1_W'],'W_4': time_dict_subcortical['task-sleep_run-2_W'],'W_5': time_dict_subcortical['task-sleep_run-3_W'],'W_6': time_dict_subcortical['task-sleep_run-4_W'],'W_7': time_dict_subcortical['task-sleep_run-5_W'],'W_8': time_dict_subcortical['task-sleep_run-6_W'],'W_9': time_dict_subcortical['task-sleep_run-7_W']}
time_dict_subcortical_1 = {'1_1': time_dict_subcortical['task-rest_run-1_1'], '1_2': time_dict_subcortical['task-rest_run-2_1'], '1_3': time_dict_subcortical['task-sleep_run-1_1'],'1_4': time_dict_subcortical['task-sleep_run-2_1'],'1_5': time_dict_subcortical['task-sleep_run-3_1'],'1_6': time_dict_subcortical['task-sleep_run-4_1'],'1_7': time_dict_subcortical['task-sleep_run-5_1'],'1_8': time_dict_subcortical['task-sleep_run-6_1'],'1_9': time_dict_subcortical['task-sleep_run-7_1']}
time_dict_subcortical_2 = {'2_1': time_dict_subcortical['task-rest_run-1_2'], '2_2': time_dict_subcortical['task-rest_run-2_2'], '2_3': time_dict_subcortical['task-sleep_run-1_2'],'2_4': time_dict_subcortical['task-sleep_run-2_2'],'2_5': time_dict_subcortical['task-sleep_run-3_2'],'2_6': time_dict_subcortical['task-sleep_run-4_2'],'2_7': time_dict_subcortical['task-sleep_run-5_2'],'2_8': time_dict_subcortical['task-sleep_run-6_2'],'2_9': time_dict_subcortical['task-sleep_run-7_2']}


parameters = [(["T1 (W)", "T2 (W)", "T3 (W)","T4 (W)","T5 (W)","T6 (W)","T7 (W)","T8 (W)","T9 (W)"], time_dict_cortical_wake, time_dict_subcortical_wake),
              (["T1 (N1)", "T2 (N1)", "T3 (N1)","T4 (N1)","T5 (N1)","T6 (N1)","T7 (N1)","T8 (N1)","T9 (N1)"], time_dict_cortical_1, time_dict_subcortical_1),
              (["T1 (N2)", "T2 (N2)", "T3 (N2)","T4 (N2)","T5 (N2)","T6 (N2)","T7 (N2)","T8 (N2)","T9 (N2)"], time_dict_cortical_2, time_dict_subcortical_2)]
# [
#     (["T1 (W)", "T1 (N1)", "T1 (N2)"], time_dict_cortical_t1, time_dict_subcortical_t1),
#     (["T2 (W)", "T2 (N1)", "T2 (N2)"], time_dict_cortical_t2, time_dict_subcortical_t2),
#     (["T3 (W)", "T3 (N1)", "T3 (N2)"], time_dict_cortical_t3, time_dict_subcortical_t3),
#     (["T4 (W)", "T4 (N1)", "T4 (N2)"], time_dict_cortical_t4, time_dict_subcortical_t4),
#     (["T5 (W)", "T5 (N1)", "T5 (N2)"], time_dict_cortical_t5, time_dict_subcortical_t5),
#     (["T6 (W)", "T6 (N1)", "T6 (N2)"], time_dict_cortical_t6, time_dict_subcortical_t6),
#     (["T7 (W)", "T7 (N1)", "T7 (N2)"], time_dict_cortical_t7, time_dict_subcortical_t7),
#     (["T8 (W)", "T8 (N1)", "T8 (N2)"], time_dict_cortical_t8, time_dict_subcortical_t8),
#     (["T9 (W)", "T9 (N1)", "T9 (N2)"], time_dict_cortical_t9, time_dict_subcortical_t9)
# ]

# Plot each combination
for i, (group_parameter, hurst_img_cortical, hurst_img_subcortical) in enumerate(parameters, start=1):
    plot_brain_regions_hurst(group_parameter, hurst_img_cortical, hurst_img_subcortical)

Visualising the transition of HE with sleep scores within a subject:

In [ ]:
def plot_subject_he(file_name,subject_name,region_name):
    df = pd.read_csv(file_name)

    sub_1_data = df[df['subject'] == subject_name]

    he_values = sub_1_data[region_name].tolist()
    runs = sub_1_data["run"].tolist()
    total_datapoints = sub_1_data["Total datapoints"].tolist()
    sleep_scores = sub_1_data["sleep_stage"].tolist()

    cumulative_sum = [sum(total_datapoints[:i+1]) for i in range(len(total_datapoints))]
    time_line = (np.array(cumulative_sum))*0.5

    # Use the indices of sleep_scores as the x-axis values
    x_values = list(range(len(time_line)))
    positive_he = np.array(he_values) > 0
    x_values = np.array(x_values)
    he_values = np.array(he_values)

    # Plotting
    plt.figure(figsize=(20, 6))  # Adjust the figure size as needed
    plt.plot(x_values, he_values, marker='o')  # 'o' for circle markers
    plt.plot(x_values[positive_he], he_values[positive_he],"--")

    # Loop through each point to add a text label
    for i, (x, y) in enumerate(zip(x_values, he_values)):
        plt.text(x, y, sleep_scores[i][0], fontsize=15, ha='right', va='bottom')
        plt.text(x, y, total_datapoints[i]*0.5, fontsize=9, ha='left', va='top')

    # Set the x-axis ticks to correspond to the indices of sleep_scores
    plt.xticks(x_values, time_line,rotation=90)
    plt.grid()
    # Adding labels and title for clarity
    plt.xlabel('Time (mins)')
    plt.ylabel('HE Values')
    plt.title(f'HE Values by Sleep Stage of {subject_name}')

    # Show the plot
    plt.show()

file_name = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected_all_datapoints.csv'
subject_name = ["sub-01","sub-03","sub-05","sub-07","sub-08","sub-09","sub-11","sub-13","sub-15","sub-17","sub-19","sub-22"]
region_name = "Left Thalamus"
for i in range(len(subject_name)):
    plot_subject_he(file_name,subject_name[i],region_name)

Box plots per total datapoints

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


def get_box_plot(W_data,_1_data ,_2_data,datapoints,samples_w,samples_1,samples_2):

    # Combine NREM1 and NREM2 data for one of the boxplots
    sleep_data = _1_data + _2_data

    plt.figure(figsize=(10, 10))

    # Plot boxplots for Wake vs Sleep
    plt.subplot(2, 1, 1)
    boxplot = plt.boxplot([W_data, sleep_data], labels=['Wake', 'Sleep'], 
                        showmeans=True, showfliers=False, meanline=True, sym='.')

    # Customize mean lines
    for mean in boxplot['means']:
        mean.set_color('green')
        mean.set_linewidth(2)
        mean.set_linestyle('--')
        x, y = mean.get_xdata(), mean.get_ydata()
        mean_value = y[0]
        plt.text(x[0], mean_value, f'mean: {mean_value:.2f}', 
                va='bottom', ha='left', color='green')  # Annotate mean value

    plt.xlabel('Subject State')
    plt.ylabel('Average Brain Hurst Exponent')
    plt.title(f'Box Plot of Average Hurst exponent by Wake vs Sleep for {(datapoints)*0.5} mins')
    plt.legend([f'Wake: {samples_w} samples , Sleep: {samples_1+samples_2} samples'])

    # Plot boxplots for Wake vs NREM1 vs NREM2
    plt.subplot(2, 1, 2)
    boxplot = plt.boxplot([W_data, _1_data, _2_data], labels=['Wake', 'NREM1', 'NREM2'], 
                        showmeans=True, showfliers=False, meanline=True, sym='.')

    # Customize mean lines
    for mean in boxplot['means']:
        mean.set_color('green')
        mean.set_linewidth(2)
        mean.set_linestyle('--')
        x, y = mean.get_xdata(), mean.get_ydata()
        mean_value = y[0]
        plt.text(x[0], mean_value, f'mean: {mean_value:.2f}', 
                va='bottom', ha='left', color='green')  # Annotate mean value

    plt.xlabel('Sleep Score')
    plt.ylabel('Average Hurst Exponent')
    plt.title(f'Box Plot of Average Hurst exponent by Wake vs NREM1 vs NREM2 for {(datapoints)*0.5} mins')
    plt.legend([f'Wake: {samples_w} samples , NREM1: {samples_1} samples, NREM2: {samples_2} samples'])

    plt.tight_layout()
    plt.show()

# Load the data
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected.csv'
df = pd.read_csv(file_path)

# Get all unique values of 'Total datapoints' column
total_datapoints_values = df['Total datapoints'].unique()

columns_to_consider = [df.columns[3]] + [df.columns[4]] + list(df.columns[5:])

# Group the DataFrame by 'sleep_stage' and 'Total datapoints'
grouped_data_w = df[df['sleep_stage'] == 'W'].groupby(['Total datapoints'])
grouped_data_1 = df[df['sleep_stage'] == '1'].groupby(['Total datapoints'])
grouped_data_2 = df[df['sleep_stage'] == '2'].groupby(['Total datapoints'])

run_freq_w = grouped_data_w['Total datapoints'].value_counts().to_dict()
run_freq_1 = grouped_data_1['Total datapoints'].value_counts().to_dict()
run_freq_2 = grouped_data_2['Total datapoints'].value_counts().to_dict()

grouped_data_w = df[df['sleep_stage'] == 'W'][columns_to_consider].groupby(['sleep_stage', 'Total datapoints']).mean()
grouped_data_1 = df[df['sleep_stage'] == '1'][columns_to_consider].groupby(['sleep_stage', 'Total datapoints']).mean()
grouped_data_2 = df[df['sleep_stage'] == '2'][columns_to_consider].groupby(['sleep_stage', 'Total datapoints']).mean()


# Filter the DataFrame for sleep stage 'W'
W_data = grouped_data_w.reset_index()
W_total_datapoints = W_data['Total datapoints'].unique()

# Filter the DataFrame for sleep stage '1'
_1_data = grouped_data_1.reset_index()
_1_total_datapoints = _1_data['Total datapoints'].unique()

# Filter the DataFrame for sleep stage '2'
_2_data = grouped_data_2.reset_index()
_2_total_datapoints = _2_data['Total datapoints'].unique()

# Compare the 'Total datapoints' values
common_total_datapoints = set(W_total_datapoints) & set(_1_total_datapoints) & set(_2_total_datapoints)

all_figures = []
# grouped_data_w.index
for i in np.sort(list(common_total_datapoints)):
    # Extract data based on sleep stage
    W_data = grouped_data_w.loc[("W",i)].tolist()
    _1_data = grouped_data_1.loc[("1",i)].tolist()
    _2_data = grouped_data_2.loc[("2",i)].tolist()
    
    get_box_plot(W_data,_1_data ,_2_data,i,run_freq_w[i],run_freq_1[i],run_freq_2[i])
    # all_figures.append(fig)

    

Box plots per subject for all datapoints

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


def get_box_plot(W_data,_1_data ,_2_data,datapoints,samples_w,samples_1,samples_2):

    # Combine NREM1 and NREM2 data for one of the boxplots
    sleep_data = _1_data + _2_data

    plt.figure(figsize=(20, 15))

    # Plot boxplots for Wake vs Sleep
    plt.subplot(2, 1, 1)
    boxplot = plt.boxplot([W_data, sleep_data], labels=['Wake', 'Sleep'], 
                        showmeans=True, showfliers=False, meanline=True, sym='.')

    # Customize mean lines
    for mean in boxplot['means']:
        mean.set_color('green')
        mean.set_linewidth(2)
        mean.set_linestyle('--')
        x, y = mean.get_xdata(), mean.get_ydata()
        mean_value = y[0]
        plt.text(x[0], mean_value, f'mean: {mean_value:.2f}', 
                va='bottom', ha='left', color='green')  # Annotate mean value

    plt.xlabel('Subject State')
    plt.ylabel('Average Brain Hurst Exponent')
    plt.title(f'Box Plot of Average Hurst exponent by Wake vs Sleep for {datapoints}')
    plt.legend([f'Wake: {samples_w} samples , Sleep: {samples_1+samples_2} samples'])

    # Plot boxplots for Wake vs NREM1 vs NREM2
    plt.subplot(2, 1, 2)
    boxplot = plt.boxplot([W_data, _1_data, _2_data], labels=['Wake', 'NREM1', 'NREM2'], 
                        showmeans=True, showfliers=False, meanline=True, sym='.')

    # Customize mean lines
    for mean in boxplot['means']:
        mean.set_color('green')
        mean.set_linewidth(2)
        mean.set_linestyle('--')
        x, y = mean.get_xdata(), mean.get_ydata()
        mean_value = y[0]
        plt.text(x[0], mean_value, f'mean: {mean_value:.2f}', 
                va='bottom', ha='left', color='green')  # Annotate mean value

    plt.xlabel('Sleep Score')
    plt.ylabel('Average Hurst Exponent')
    plt.title(f'Box Plot of Average Hurst exponent by Wake vs NREM1 vs NREM2 for {datapoints}')
    plt.legend([f'Wake: {samples_w} samples , NREM1: {samples_1} samples, NREM2: {samples_2} samples'])

    plt.tight_layout()
    plt.show()

# Load the data
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_corrected.csv'
df = pd.read_csv(file_path)

# Get all unique values of 'Total datapoints' column
total_datapoints_values = df['subject'].unique()

columns_to_consider = [df.columns[0]] +[df.columns[3]] + list(df.columns[5:])

# Group the DataFrame by 'sleep_stage' and 'Total datapoints'
grouped_data_w = df[df['sleep_stage'] == 'W'].groupby(['subject'])
grouped_data_1 = df[df['sleep_stage'] == '1'].groupby(['subject'])
grouped_data_2 = df[df['sleep_stage'] == '2'].groupby(['subject'])

run_freq_w = grouped_data_w['subject'].value_counts().to_dict()
run_freq_1 = grouped_data_1['subject'].value_counts().to_dict()
run_freq_2 = grouped_data_2['subject'].value_counts().to_dict()

grouped_data_w = df[df['sleep_stage'] == 'W'][columns_to_consider].groupby(["sleep_stage","subject"]).mean()
grouped_data_1 = df[df['sleep_stage'] == '1'][columns_to_consider].groupby(['sleep_stage', 'subject']).mean()
grouped_data_2 = df[df['sleep_stage'] == '2'][columns_to_consider].groupby(['sleep_stage', 'subject']).mean()


# Filter the DataFrame for sleep stage 'W'
W_data = grouped_data_w.reset_index()
W_total_datapoints = W_data['subject'].unique()

# Filter the DataFrame for sleep stage '1'
_1_data = grouped_data_1.reset_index()
_1_total_datapoints = _1_data['subject'].unique()

# Filter the DataFrame for sleep stage '2'
_2_data = grouped_data_2.reset_index()
_2_total_datapoints = _2_data['subject'].unique()

# Compare the 'Total datapoints' values
common_total_datapoints = set(W_total_datapoints) & set(_1_total_datapoints) & set(_2_total_datapoints)

all_figures = []
# grouped_data_w.index
for i in np.sort(list(common_total_datapoints)):
    # Extract data based on sleep stage
    W_data = grouped_data_w.loc[("W",i)].tolist()
    _1_data = grouped_data_1.loc[("1",i)].tolist()
    _2_data = grouped_data_2.loc[("2",i)].tolist()
    
    get_box_plot(W_data,_1_data ,_2_data,i,run_freq_w[i],run_freq_1[i],run_freq_2[i])
    # all_figures.append(fig)

    

Box plots based on time

In [ ]:
file_path='../output_directory/evaluation_dataset/dfa_hurst_dataset_roi.csv'
df = pd.read_csv(file_path)

#Timeline = T1
W_data_1 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-rest_run-1')]['AWB'].tolist()
N1_data_1 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-rest_run-1')]['AWB'].tolist()
N2_data_1 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-rest_run-1')]['AWB'].tolist()
data_1 = W_data_1 + N1_data_1 + N2_data_1

#Timeline = T2
W_data_2 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-rest_run-2')]['AWB'].tolist()
N1_data_2 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-rest_run-2')]['AWB'].tolist()
N2_data_2 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-rest_run-2')]['AWB'].tolist()
data_2 = W_data_2 + N1_data_2 + N2_data_2

#Timeline = T3
W_data_3 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-1')]['AWB'].tolist()
N1_data_3 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-1')]['AWB'].tolist()
N2_data_3 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-1')]['AWB'].tolist()
data_3 = W_data_3 + N1_data_3 + N2_data_3

#Timeline = T4
W_data_4 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-2')]['AWB'].tolist()
N1_data_4 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-2')]['AWB'].tolist()
N2_data_4 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-2')]['AWB'].tolist()
data_4 = W_data_4 + N1_data_4 + N2_data_4

#Timeline = T5
W_data_5 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
N1_data_5 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
N2_data_5 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
data_5 = W_data_5 + N1_data_5 + N2_data_5

#Timeline = T6
W_data_6 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
N1_data_6 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
N2_data_6 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-3')]['AWB'].tolist()
data_6 = W_data_6 + N1_data_6 + N2_data_6

#Timeline = T7
W_data_7 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-4')]['AWB'].tolist()
N1_data_7 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-4')]['AWB'].tolist()
N2_data_7 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-4')]['AWB'].tolist()
data_7 = W_data_7 + N1_data_7 + N2_data_7

#Timeline = T8
W_data_8 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-5')]['AWB'].tolist()
N1_data_8 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-5')]['AWB'].tolist()
N2_data_8 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-5')]['AWB'].tolist()
data_8 = W_data_8 + N1_data_8 + N2_data_8

#Timeline = T9
W_data_9 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-6')]['AWB'].tolist()
N1_data_9 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-6')]['AWB'].tolist()
N2_data_9 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-6')]['AWB'].tolist()
data_9 = W_data_9 + N1_data_9 + N2_data_9

#Timeline = T10
W_data_10 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-7')]['AWB'].tolist()
N1_data_10 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-7')]['AWB'].tolist()
N2_data_10 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-7')]['AWB'].tolist()
data_10 = W_data_10 + N1_data_10 + N2_data_10


#Timeline = T11
W_data_11 = df[(df['sleep_stage'] == 'W') & (df['run'] == 'task-sleep_run-8')]['AWB'].tolist()
N1_data_11 = df[(df['sleep_stage'] == '1') & (df['run'] == 'task-sleep_run-8')]['AWB'].tolist()
N2_data_11 = df[(df['sleep_stage'] == '2') & (df['run'] == 'task-sleep_run-8')]['AWB'].tolist()
data_11 = W_data_11 + N1_data_11 + N2_data_11

median_values=[]

plt.figure(figsize=(20, 10))
# boxplot=plt.boxplot([W_data_1, N1_data_1,N2_data_1,W_data_2, N1_data_2,N2_data_2,W_data_3, N1_data_3,N2_data_3,W_data_4, N1_data_4,N2_data_4,W_data_5, N1_data_5,N2_data_5,W_data_6, N1_data_6,N2_data_6,W_data_7, N1_data_7,N2_data_7,W_data_8, N1_data_8,N2_data_8,W_data_9, N1_data_9,N2_data_9,W_data_10, N1_data_10,N2_data_10,W_data_11, N1_data_11,N2_data_11], labels=['T1']*3 + ['T2']*3+ ['T3']*3 + ['T4']*3+['T5']*3 + ['T6']*3+['T7']*3 + ['T8']*3+['T9']*3 + ['T10']*3+['T11']*3,showmeans=False, showfliers=False, sym='.')
boxplot=plt.boxplot([data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11], labels=['T1'] + ['T2']+ ['T3'] + ['T4']+['T5'] + ['T6']+['T7'] + ['T8']+['T9'] + ['T10']+['T11'],showmeans=False, showfliers=False, sym='.')

for median in boxplot['medians']:
    median.set_color('red')  
    median.set_linewidth(2)  
    x, y = median.get_xdata(), median.get_ydata()
    median_values.append(y[0])  
    plt.text(x[0], y[0], f'{y[0]:.2f}', va='bottom', ha='left', color='red') 


# plt.plot([1, 4,7,10,13,16,19,22,25,28,31], median_values[::3], color='blue', linestyle='--')
# plt.plot([2, 5,8,11,14,17,20,23,26,29,32], median_values[1::3], color='green', linestyle='--')
# plt.plot([3, 6,9,12,15,18,21,24,27,30,33], median_values[2::3], color='orange', linestyle='--')

plt.plot(np.arange(1, 12),median_values, color='blue', linestyle='--')

plt.xlabel('Subject State')
plt.ylabel('Average Brain Hurst Exponent')
plt.title('Box Plot of Average Hurst Exponent vs Time')
plt.show()


Vionlin Plot for showing transition from Wake to Sleep per subject.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def get_violin_plot(transitions,transitions_melted,parameter_1, parameter_2):
    # Create the violin plot with seaborn
    sns.violinplot(data=transitions_melted, x='Condition', y='Hurst_Exponent',inner="point",saturation=0.4)

    # Draw lines connecting the paired points for each subject
    for i in range(len(transitions)):
        if transitions.iloc[i][parameter_1] > transitions.iloc[i][parameter_2]:
            color = 'green'
        else:
            color = 'red'
        
        # Plot the line with the determined color
        plt.plot([parameter_1, parameter_2], 
                [transitions.iloc[i][parameter_1], transitions.iloc[i][parameter_2]],
                color=color, alpha=1)
    plt.ylim(0.4, 1.2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_roi_corrected.csv'  # Replace with your actual file path
hurst_data = pd.read_csv(file_path)

# Averaging across all the regions to obtain a single Hurst exponent for each observation
hurst_columns = hurst_data.columns[5:]

# Preparing data for wake and sleep states
wake_data = hurst_data[hurst_data['sleep_stage'] == 'W']
nrem1_data = hurst_data[hurst_data['sleep_stage'] == '1']
nrem2_data = hurst_data[hurst_data['sleep_stage'] == '2']
sleep_data = pd.concat([nrem1_data,nrem2_data])


# Find the common subjects across Wake, NREM1, and NREM2
common_subjects = set(wake_data['subject']) & set(sleep_data['subject'])

# Filter the original dataframes to only include these common subjects
wake_data_common = wake_data[wake_data['subject'].isin(sorted(common_subjects))]
sleep_data_common = sleep_data[sleep_data['subject'].isin(sorted(common_subjects))]


wake_grouped = wake_data_common.groupby(['Total datapoints',"subject"])['AWB'].mean().reset_index()
sleep_grouped = sleep_data_common.groupby(['Total datapoints',"subject"])['AWB'].mean().reset_index()

# Set up the matplotlib figure
plt.figure(figsize=(20, 8))
# Final touches on the plot
plt.title('Transition of Subjects\' Hurst Exponents from Wake to Sleep')

# Merging the wake and sleep dataframes
transitions_WS = pd.merge(wake_grouped, sleep_grouped, on='subject', suffixes=('_Wake', '_Sleep'))

transitions_WS = transitions_WS[transitions_WS["Total datapoints_Wake"]==transitions_WS["Total datapoints_Sleep"]]
# Melt the DataFrame to make it suitable for seaborn's violinplot function
transitions_WS_melted = pd.melt(transitions_WS, id_vars=['subject'], value_vars=['AWB_Wake', 'AWB_Sleep'],
                             var_name='Condition', value_name='Hurst_Exponent')

transitions_WS["Difference"] = transitions_WS["AWB_Wake"] - transitions_WS["AWB_Sleep"]
get_violin_plot(transitions_WS,transitions_WS_melted,"AWB_Wake", "AWB_Sleep")

print(common_subjects)
transitions_WS

Vionlin Plot for showing transition from Wake to NREM 1 and NREM 2 per subject.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
file_path = '../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_roi_corrected.csv'  # Replace with your actual file path
hurst_data = pd.read_csv(file_path)

# Averaging across all the regions to obtain a single Hurst exponent for each observation
hurst_columns = hurst_data.columns[5:]
hurst_data['Mean_Hurst_Exponent'] = hurst_data[hurst_columns].mean(axis=1)

# Preparing data for wake and sleep states
wake_data = hurst_data[hurst_data['sleep_stage'] == 'W']
nrem1_data = hurst_data[hurst_data['sleep_stage'] == '1']
nrem2_data = hurst_data[hurst_data['sleep_stage'] == '2']
sleep_data = pd.concat([nrem1_data,nrem2_data])


# Find the common subjects across Wake, NREM1, and NREM2
common_subjects = set(wake_data['subject']) & set(nrem1_data['subject']) & set(nrem2_data['subject'])

# Filter the original dataframes to only include these common subjects
wake_data_common = wake_data[wake_data['subject'].isin(sorted(common_subjects))]
nrem1_data_common = nrem1_data[nrem1_data['subject'].isin(sorted(common_subjects))]
nrem2_data_common = nrem2_data[nrem2_data['subject'].isin(sorted(common_subjects))]


wake_grouped = wake_data_common.groupby(['Total datapoints',"subject"])['AWB'].mean().reset_index()
nrem1_grouped = nrem1_data_common.groupby(['Total datapoints',"subject"])['AWB'].mean().reset_index()
nrem2_grouped = nrem2_data_common.groupby(['Total datapoints',"subject"])['AWB'].mean().reset_index()


In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(20, 8))
# Final touches on the plot

plt.title('Transition of Subjects\' Hurst Exponents from Wake to NREM1')

# Merging the wake and sleep dataframes
transitions_W1 = pd.merge(wake_grouped, nrem1_grouped, on='subject', suffixes=('_Wake', '_NREM1'))
transitions_W1 = transitions_W1[transitions_W1["Total datapoints_Wake"]==transitions_W1["Total datapoints_NREM1"]]

# Melt the DataFrame to make it suitable for seaborn's violinplot function
transitions_W1_melted = pd.melt(transitions_W1, id_vars=['subject'], value_vars=['AWB_Wake', 'AWB_NREM1'],
                             var_name='Condition', value_name='Hurst_Exponent')

transitions_W1["Difference"] = transitions_W1["AWB_Wake"] - transitions_W1["AWB_NREM1"]

get_violin_plot(transitions_W1,transitions_W1_melted,"AWB_Wake", "AWB_NREM1")

transitions_W1

In [ ]:
plt.figure(figsize=(20, 8))
# Final touches on the plot
plt.title('Transition of Subjects\' Hurst Exponents from Wake to NREM2')
# Merging the wake and sleep dataframes
transitions_W2 = pd.merge(wake_grouped, nrem2_grouped, on='subject', suffixes=('_Wake', '_NREM2'))
transitions_W2 = transitions_W2[transitions_W2["Total datapoints_Wake"]==transitions_W2["Total datapoints_NREM2"]]

# Melt the DataFrame to make it suitable for seaborn's violinplot function
transitions_W2_melted = pd.melt(transitions_W2, id_vars=['subject'], value_vars=['AWB_Wake', 'AWB_NREM2'],
                             var_name='Condition', value_name='Hurst_Exponent')

transitions_W2["Difference"] = transitions_W2["AWB_Wake"] - transitions_W2["AWB_NREM2"]

get_violin_plot(transitions_W2,transitions_W2_melted,"AWB_Wake", "AWB_NREM2")

transitions_W2

In [ ]:
plt.figure(figsize=(20, 8))
# Final touches on the plot
plt.title('Transition of Subjects\' Hurst Exponents from NREM1 to NREM2')
# Merging the wake and sleep dataframes
transitions_12 = pd.merge(nrem1_grouped, nrem2_grouped, on='subject', suffixes=('_NREM1', '_NREM2'))
transitions_12 = transitions_12[transitions_12["Total datapoints_NREM1"]==transitions_12["Total datapoints_NREM2"]]

# Melt the DataFrame to make it suitable for seaborn's violinplot function
transitions_12_melted = pd.melt(transitions_12, id_vars=['subject'], value_vars=['AWB_NREM1', 'AWB_NREM2'],
                             var_name='Condition', value_name='Hurst_Exponent')

transitions_12["Difference"] = transitions_12["AWB_NREM1"] - transitions_12["AWB_NREM2"]

get_violin_plot(transitions_12,transitions_12_melted,"AWB_NREM1", "AWB_NREM2")

transitions_12

In [ ]:
import pandas as pd
from scipy import stats

t_stat_WS, p_value_WS = stats.ttest_rel(transitions_WS['AWB_Wake'], transitions_WS['AWB_Sleep'])
t_stat_W1, p_value_W1 = stats.ttest_rel(transitions_W1['AWB_Wake'], transitions_W1['AWB_NREM1'])
t_stat_W2, p_value_W2 = stats.ttest_rel(transitions_W2['AWB_Wake'], transitions_W2['AWB_NREM2'])
t_stat_12, p_value_12 = stats.ttest_rel(transitions_12['AWB_NREM1'], transitions_12['AWB_NREM2'])

print(f"Transition from wake to sleep : T-statistic: {t_stat_WS}, P-value: {p_value_WS}")
print(f"Transition from wake to NREM1 : T-statistic: {t_stat_W1}, P-value: {p_value_W1}")
print(f"Transition from wake to NREM2 : T-statistic: {t_stat_W2}, P-value: {p_value_W2}")
print(f"Transition from NREM1 to NREM2 : T-statistic: {t_stat_12}, P-value: {p_value_12}")

Statistical Testing:
    - t-testing for Wake vs Sleep
    - Anova test for Wake vs N1 vs N2

In [8]:
from t_test import evaluate_t_test
from anova_test import evaluate_anova_test

evaluate_t_test('../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_roi_corrected.csv', fdr_corrected = False, save_results = False)
evaluate_anova_test('../output_directory/evaluation_dataset/hurst_evaluation/dfa_hurst_dataset_roi_corrected.csv', fdr_corrected = False, save_results = False)

ModuleNotFoundError: No module named 't_test'

Linear Mixed Effect Model

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

df = pd.read_csv('../output_directory/evaluation_dataset/dfa_hurst_dataset_roi.csv')


df['sleep_stage'] = df['sleep_stage'].astype('category')
df['sleep_stage'] = df['sleep_stage'].cat.reorder_categories(['W', '1', '2', '3'], ordered=True)
df = df.rename(columns={col: col.replace(' ', '_') for col in df.columns})

formula = 'AWB ~ sleep_stage + run'
random_formula = '0 + run'

# Fit the Linear Mixed-Effects Model
model = smf.mixedlm(formula, data=df, groups=df['sleep_stage'])
result = model.fit()

# df.head()
print(result.summary())

# Extract the coefficients from the model results
# coefficients = result.params
# # print(coefficients)
# # Plot the coefficients
# plt.figure(figsize=(20,5))
# sns.barplot(x=coefficients.index, y=coefficients.values)
# plt.xlabel('Coefficient')
# plt.ylabel('Value')
# plt.title('Coefficients of Linear Mixed-Effects Model')
# plt.xticks(rotation=45)
# plt.show()

p_values = result.pvalues

# Step 2: Prepare the data for plotting
predictor_variables = p_values.index

# Step 3: Create a plot to visualize the p-values
plt.figure(figsize=(10, 6))
plt.barh(predictor_variables, p_values, color='skyblue')
plt.xlabel('P-values')
plt.ylabel('Predictor Variables')
plt.title('P-values of Predictor Variables')
plt.axvline(x=0.05, color='red', linestyle='--', linewidth=1)  # Add a line at alpha = 0.05
plt.gca().invert_yaxis()  # Invert y-axis to have the highest p-values at the top
plt.show()


predicted_values = result.fittedvalues

# Create scatter plots
plt.figure(figsize=(10, 6))
plt.scatter(df['AWB'], predicted_values, color='blue', alpha=0.5)
plt.xlabel('Observed AWB')
plt.ylabel('Predicted AWB')
plt.title('Observed vs. Predicted AWB')

# Add a diagonal line for reference
plt.plot([min(df['AWB']), max(df['AWB'])], [min(df['AWB']), max(df['AWB'])], color='red', linestyle='--')

plt.show()

Connectivity Matrix
- Time series 
- Hurst Exponent
- Variance

In [ ]:
from nilearn.connectome import ConnectivityMeasure

def get_connectivity_matrix(file_path):

    # Fetch cortical atlas
    cortical_atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    cortical_maps = cortical_atlas.maps
    cortical_labels = cortical_atlas.labels
    cortical_masker = maskers.NiftiLabelsMasker(labels_img=cortical_maps, standardize=True)

    # Fetch subcortical atlas
    subcortical_atlas = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
    subcortical_maps = subcortical_atlas.maps
    subcortical_labels = subcortical_atlas.labels
    subcortical_masker = maskers.NiftiLabelsMasker(labels_img=subcortical_maps, standardize=True)
    
    combined_labels = cortical_labels[1:] + subcortical_labels[1:]  # Skip 'Background' label from subcortical labels

    # Apply masker
    time_series_cortical = cortical_masker.fit_transform(file_path)
    time_series_subcortical = subcortical_masker.fit_transform(file_path)
    time_series_combined = np.concatenate((time_series_cortical, time_series_subcortical), axis=1)

    # Compute correlation matrix
    correlation_measure = ConnectivityMeasure(kind="correlation", standardize="zscore_sample")
    correlation_matrix = correlation_measure.fit_transform([time_series_combined])[0]

    # Set diagonal to zero
    np.fill_diagonal(correlation_matrix, 0)

    ax = plotting.plot_matrix(correlation_matrix, labels=combined_labels,cmap=plt.cm.RdBu_r, colorbar=True,title="Correlation matrix of ROIs with connectivity measure using correlation")
    plt.gcf().set_size_inches(20, 20)
    plt.show()

    # plt.figure(figsize=(10, 8))
    # plt.imshow(correlation_matrix, cmap='viridis', interpolation='nearest')
    # plt.colorbar(label='Correlation strength')
    # plt.title('Functional Connectome')
    # plt.xlabel('ROIs')
    # plt.ylabel('ROIs')
    # plt.show()

    plt.show()

# Example usage
get_connectivity_matrix(file_path="../output_directory/sub-05/func/sub-05_task-rest_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_postproc_smooth.nii.gz")
get_connectivity_matrix(file_path="../output_directory/sub-03/func/sub-03_task-sleep_run-5_space-MNI152NLin2009cAsym_desc-preproc_bold_postproc_smooth.nii.gz")


In [ ]:
import seaborn as sns


def calculate_mean(file_path, parameter_index):
    
    df = pd.read_csv(file_path)

    columns_to_consider = [df.columns[parameter_index]] + list(df.columns[5:])

    # Filter out zero values
    non_zero_df = df[df[columns_to_consider] != 0]

    # Group by parameter and calculate the mean for each region
    mean_non_zero_hurst_exponents = non_zero_df.groupby(df.columns[parameter_index], as_index=False).mean()

    return mean_non_zero_hurst_exponents


def compute_connectivity(hurst_exponents):
    num_regions = len(hurst_exponents)
    connectivity_matrix = np.zeros((num_regions, num_regions))
    for i in range(num_regions):
        for j in range(i + 1, num_regions):  # Only compute upper triangle (excluding diagonal)
            connectivity_matrix[i, j] = abs(hurst_exponents[i] - hurst_exponents[j])
            connectivity_matrix[j, i] = connectivity_matrix[i, j]  # Symmetric matrix
    return connectivity_matrix



Hurst Connectivity Matrix

In [ ]:
file_path = '../output_directory/evaluation_dataset/dfa_hurst_dataset.csv'
mean_hurst_per_sleep_stage = calculate_mean(file_path, 3)
sleep_stage_dict_cortical = {}
sleep_stage_dict_subcortical = {}

sleep_scores = ['W','1', '2']

for score in sleep_scores:
    # Filter the DataFrame for the current sleep score
    score_df = mean_hurst_per_sleep_stage[mean_hurst_per_sleep_stage['sleep_stage'] == score]

    # Extract connectivity data for the current sleep score
    connectivity_data = score_df.iloc[:, 5:].values  # Assuming connectivity data starts from column index 5
    hurst_connectivity = compute_connectivity(connectivity_data[0])
    # Plot the connectivity matrix
    plt.figure(figsize=(14, 10))
    sns.heatmap(hurst_connectivity, cmap='coolwarm', xticklabels=score_df.columns[5:], yticklabels=score_df.columns[5:])
    plt.title(f'Connectivity Matrix for hurst exponent Sleep Score {score}')
    plt.xlabel('Brain Regions')
    plt.ylabel('Brain Regions')
    plt.xticks(rotation=90)
    plt.show()

Variance Connectivity Matrix

In [ ]:
file_path = '../output_directory/evaluation_dataset/variance_dataset.csv'
mean_hurst_per_sleep_stage = calculate_mean(file_path, 3)
sleep_stage_dict_cortical = {}
sleep_stage_dict_subcortical = {}

sleep_scores = ['W','1', '2']

for score in sleep_scores:
    # Filter the DataFrame for the current sleep score
    score_df = mean_hurst_per_sleep_stage[mean_hurst_per_sleep_stage['sleep_stage'] == score]

    # Extract connectivity data for the current sleep score
    connectivity_data = score_df.iloc[:, 5:].values  # Assuming connectivity data starts from column index 5
    hurst_connectivity = compute_connectivity(connectivity_data[0])
    # Plot the connectivity matrix
    plt.figure(figsize=(14, 10))
    sns.heatmap(hurst_connectivity, cmap='coolwarm', xticklabels=score_df.columns[5:], yticklabels=score_df.columns[5:])
    plt.title(f'Connectivity Matrix for Vraiance of Sleep Score {score}')
    plt.xlabel('Brain Regions')
    plt.ylabel('Brain Regions')
    plt.xticks(rotation=90)
    plt.show()